In [36]:
%load_ext autoreload
%autoreload 2

from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

# Second dimension of the feature is dim2
feature_dim_2 = 11

# Save data to array file first
save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 50
batch_size = 100
verbose = 1
num_classes = 5

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)


Saving vectors of label - 'other':   0%|          | 0/131 [00:00<?, ?it/s]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



Saving vectors of label - 'other': 100%|██████████| 131/131 [03:59<00:00,  1.82s/it]

Saving vectors of label - 'bed':  76%|███████▋  | 1308/1713 [00:05<00:01, 229.75it/s]


Saving vectors of label - 'Chopin': 100%|██████████| 104/104 [03:08<00:00,  1.82s/it]

Saving vectors of label - '.ipynb_checkpoints': 0it [00:00, ?it/s]

Saving vectors of label - 'cat':  84%|████████▍ | 1456/1733 [00:06<00:01, 229.10it/s]


Saving vectors of label - 'happy': 100%|██████████| 1742/1742 [00:07<00:00, 231.03it/s]

In [41]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

# Building The Model Then Training it

In [42]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Train on 3253 samples, validate on 2170 samples
Epoch 1/50
3253/3253 [==============================] - 2s 761us/step - loss: 1.7573 - acc: 0.4196 - val_loss: 0.9139 - val_acc: 0.6355
Epoch 2/50
3253/3253 [==============================] - 2s 480us/step - loss: 0.8863 - acc: 0.6575 - val_loss: 0.5843 - val_acc: 0.7728
Epoch 3/50
3253/3253 [==============================] - 2s 476us/step - loss: 0.6478 - acc: 0.7698 - val_loss: 0.4368 - val_acc: 0.8419
Epoch 4/50
3253/3253 [==============================] - 2s 462us/step - loss: 0.5326 - acc: 0.8085 - val_loss: 0.4087 - val_acc: 0.8493
Epoch 5/50
3253/3253 [==============================] - 1s 447us/step - loss: 0.4432 - acc: 0.8451 - val_loss: 0.4730 - val_acc: 0.8175
Epoch 6/50
3253/3253 [==============================] - 1s 459us/step - loss: 0.3967 - acc: 0.8657 - val_loss: 0.2893 - val_acc: 0.9032
Epoch 7/50
3253/3253 [==============================] - 1s 456us/step - loss: 0.3144 - acc: 0.8909 - val_loss: 0.2801 - val_acc: 0.8972


## Prediction

In [52]:
print(predict('./test/other_test (6).mp3', model=model))

Chopin


In [54]:
import glob
for i in glob.glob('./test/*.mp3'):
    print("{0} is {1}".format(i,predict(i, model=model)))

./test/other_test (1).mp3 is other
./test/other_test (3).mp3 is Chopin
./test/Kevin Kern - wave.mp3 is Chopin
./test/other_test (2).mp3 is other
./test/Kevin Kern - We Should Waltz.mp3 is Chopin
./test/other_test (6).mp3 is Chopin
./test/other_test (4).mp3 is Chopin
./test/other_test (8).mp3 is Chopin
./test/other_test (5).mp3 is bed
./test/Kevin Kern - Le Jardin.mp3 is Chopin
